In [7]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm
from glob import glob
import torchaudio
import pandas as pd

In [22]:
df = pd.read_csv("/home/suisei/Code/copy/s3prl/s3prl/downstream/separation_stft2/datasets/Libri2Mix/wav16k/min/test/mix_clean/wav.scp", sep=" ", header=None)


In [24]:
count = 0
for i, path in enumerate(df.iloc[:, 1]):
    # count += torchaudio.info(i).num_frames/16000
    # count += os.path.getsize(i)
    shutil.copy(path, os.path.join("/media/suisei/DataDisk/DLHLP/data/tmp", str(i)+".wav"))

In [18]:
count/1024/1024/1024

0.13265874

In [62]:
df = pd.read_csv("/home/suisei/Code/copy/s3prl/s3prl/downstream/voxceleb1/veri_test_class.txt", sep=" ", header=None)

In [63]:
count = 0
length_list = []
split_list = []
path_list = []
spk_list = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    split, wav_path = row
    path = os.path.join("/media/suisei/DataDisk/DLHLP/data/voxceleb1/*/wav", wav_path.strip())
    try:
        length_list.append(torchaudio.info(glob(path)[0]).num_frames/16000)
        split_list.append(split)
        path_list.append(wav_path.strip())
        spk_list.append(wav_path.split("/")[0])
    except:
        pass

100%|██████████| 153516/153516 [00:08<00:00, 18913.91it/s]


In [64]:
new_df = pd.DataFrame({"split": split_list, "path": path_list, "length": length_list, "spk": spk_list})

In [65]:
train_df = new_df[new_df["split"] == 1]
dev_df = new_df[new_df["split"] == 2]
test_df = new_df[new_df["split"] == 3]

In [7]:
train_df.length.sum()/3600

318.41221321180547

In [8]:
len(train_df), len(dev_df), len(test_df)

(138361, 6904, 8251)

In [24]:
len(dev_df.spk.unique())

1251

In [66]:
part_train_df = train_df.groupby('spk', group_keys=False).apply(lambda x: x.sample(11, random_state=42))
# part_dev_df = dev_df[dev_df.spk!='id10842'].groupby('spk', group_keys=False).apply(lambda x: x.sample(2, random_state=42))

In [69]:
part_train_df.shape

(13761, 4)

In [68]:
part_train_df.spk.unique().__len__()

1251

In [16]:
new_total_df = pd.concat([part_train_df, dev_df, test_df]).reset_index(drop=True)

In [17]:
new_total_df[["split", "path"]].to_csv("/home/suisei/Code/copy/s3prl/s3prl/downstream/voxceleb1/mini_veri_test_class.txt", sep=" ", header=None, index=False)

In [23]:
count/3600/16000

10.24116453125

In [2]:
src_dir = "/media/suisei/DataDisk/DLHLP/data/voxceleb1"
target_dir = "/media/suisei/DataDisk/DLHLP/data/mini_vox"

In [3]:
with open("/home/suisei/Code/copy/s3prl/s3prl/downstream/voxceleb1/veri_test_class.txt", "r") as f:
    wav_paths = f.readlines()

In [6]:
for a in tqdm(wav_paths):
    split, wav_path = a.split(" ")
    if split == "3":
        wav_path = wav_path.strip()
        src_path = glob(f"{src_dir}/*/wav/{wav_path}")[0]#os.path.join(src_dir, wav_path)
        target_path = src_path.replace("voxceleb1", "mini_vox") #os.path.join(target_dir, wav_path)
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        shutil.copy(src_path, target_path)
    

100%|██████████| 153516/153516 [00:05<00:00, 25952.23it/s]


In [99]:
for split in ["train_1000", "dev_300", "test"]:
    for cond in ["clean", "noisy"]:
        with open(f"/home/suisei/Code/copy/s3prl/s3prl/downstream/enhancement_stft2/datasets/voicebank/wav16k/{split}/{cond}/wav.scp", "r") as f:
            wav_paths = f.readlines()
            for a in tqdm(wav_paths):
                src_path = a.split(" ")[1]
                src_path = src_path.strip()
                target_path = src_path.replace("noisy-vctk-16k", "mini-vctk")
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                shutil.copy(src_path, target_path)

100%|██████████| 824/824 [00:00<00:00, 2065.09it/s]


In [102]:
for split in ["train_1400", "dev_1000", "test"]:
    for cond in ["mix_clean", "noise", "s1", "s2"]:
        with open(f"/home/suisei/Code/copy/s3prl/s3prl/downstream/separation_stft2/datasets/Libri2Mix/wav16k/min/{split}/{cond}/wav.scp", "r") as f:
            wav_paths = f.readlines()
            for a in tqdm(wav_paths):
                src_path = a.split(" ")[1]
                src_path = src_path.strip()
                target_path = src_path.replace("Libri2Mix", "mini_libri2mix")
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                shutil.copy(src_path, target_path)

100%|██████████| 3000/3000 [00:01<00:00, 1878.41it/s]
